<a href="https://colab.research.google.com/github/vatsaaa/Flowise/blob/main/semester_1/03_assignments/mfml/Assignment_01_Q01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python function to create a random matrix, but without using libraries

### Copyright (c) 2023 G. Ankur Vatsa
This code is for my M. Tech. assignments. The copyright is addded in good faith that it will restrict others from using and copying for submitting as their own assignments.

The algorithms implemented in the code below are all well known. I have explained the algorithms in the text before the code, which you are welcome to read and follow. Thus, if you happen to walk into this repository, I request you not copy or use this code.

In [56]:
import random

def generate_random_matrix(rows: int, columns: int, allow_decimals: bool=False) -> list:
    matrix = []

    for _ in range(rows):
      if allow_decimals:
        # Chose 100 and 990 so that the matrix printing is better to look at
        row = [random.choice([random.randint(10000, 99999),
                              round(random.uniform(0, 1), 2)])
                              for _ in range(columns)]
      else:
        # Chose 100 and 990 so that the matrix printing is better to look at
        row = [random.randint(10000, 99999) for _ in range(columns)]

        matrix.append(row)

    return matrix

In [29]:
three_by_three = generate_random_matrix(3, 3, False)

three_by_three

[[618, 909, 905], [504, 809, 196], [386, 899, 921]]

By defaut, the matrix generated is printed in a single line, that really does not look like a matrix. So, I have created my own function to print a matrix that looks like a matrix.


```
[ [ 584, 108], [118, 465 ] ]    Vs    [ [ 584, 108 ],
                                        [ 118, 465 ] ]
```

The `print_matrix` function takes a matrix as input and prints it in a formatted way:

- print the opening square bracket "[" to begin the matrix
- iterate over each row in the matrix, and use `enumerate` function to get both the index (i) and the row (row) in each iteration.
- for each iteration compress the lists to create an updated row.
- If the absolute value in matrix-cell is less than `sys.float_info.epsilon`, the element is replaced with 0. This is done to handle floating-point precision issues and ensure that very small values are treated as 0.
- Finally, a closing square bracket "]" is printed to represent the end of the matrix.

In summary, the print_matrix function takes a matrix, replaces very small values with 0, and prints it.

In [5]:
import sys

def print_matrix(matrix):
    print("[", end="")
    for i, row in enumerate(matrix):
        updated_row = [0 if abs(element) < sys.float_info.epsilon else element for element in row]
        print(updated_row, end="")
        if i != len(matrix) - 1:
            print(",\n", end="")
    print("]")


In [30]:
print_matrix(three_by_three)

[[618, 909, 905],
[504, 809, 196],
[386, 899, 921]]


Now we get a two_by_one matrix, before creating the augmented matrix

In [31]:
three_by_one = generate_random_matrix(3, 1, False)

print_matrix(three_by_one)

[[407],
[216],
[398]]


In the well known Gauss Elimination, the augmented matrix is created by keeping `lhs_A | rhs_b` together to create a single matrix. `construct_augmented_matrix()` function creates such an augmented matrix.

Length of matrix A must be equal to the length of matrix b. If they are not equal, it raises a generic `Exception` with proper message that clarifies the input expected by the function.

Following this basic check, for every row in lhs_A and rhs_b the function bunches together `row` and `bi` in a single row of  new matrix and returns the final matrix thus created.


In [32]:
def construct_augmented_matrix(lhs_A: list, rhs_b: list):
    if len(lhs_A) != len(rhs_b):
        raise Exception("Number of rows in A must be equal to the length of b")

    # Zip lhs_A and rhs_b to create the augmented matrix
    augmented_matrix = [row + bi for row, bi in zip(lhs_A, rhs_b)]

    return augmented_matrix

In [33]:
A = three_by_three
b = three_by_one

augmented_matrix_A_b = construct_augmented_matrix(A, b)

print_matrix(augmented_matrix_A_b)

[[618, 909, 905, 407],
[504, 809, 196, 216],
[386, 899, 921, 398]]


Function to find row echelon form of a given matrix

The `find_row_echelon_form()` function takes a matrix as input and returns the row echelon form of the given matrix. Row echelon form is a specific form of a matrix where each row has more leading zeros than the row above it.

- Get the number of rows in the matrix using the len function, and number columns is the length of one row
- Initialize the `pivot` variable, to keep track of the current pivot position. `pivot position` is the column index where the next pivot element should be found.

Now, iterate over each row in the matrix
- Check if the `pivot position` is greater than or equal to the number of columns. If it is equal to the number of columns, break out of the loop to stop the algorithm when it processes the last column.
- Check if the element at the current row and pivot position is zero. If it is zero, it means that a pivot element needs to be found in a lower row. In this case, iterate over the rows below the current row using another for loop.
- Inside the inner loop, check if the element at the current row and pivot position is non-zero. If it is non-zero, swap the current row with the row that has the non-zero element. This step ensures that a non-zero pivot element is brought to the current row.
- If no non-zero element is found in the rows below the current row, increment the pivot position and continue to the next iteration of the outer loop. This step ensures that the algorithm moves to the next column when there are no non-zero elements below the current row.
- If a non-zero element is found in the rows below the current row, perform elementary row transformations to make all elements below the pivot position zero. This step ensures that below the pivot, we get zeros in the entire column.
- Inside the inner loop, calculate the factor by dividing the element at the current row and pivot position by the element at the pivot row and pivot position. This factor will be used to eliminate the non-zero element below the pivot.
- Iterate over the columns starting from the pivot position and update the elements in the current row by subtracting the product of the factor and the corresponding element in the pivot row. This step ensures that all elements below the pivot position become zero.
- Increment the pivot position and continue to the next iteration of the outer loop. This step ensures that the algorithm moves to the next column.

As the outer loop finishes, the modified matrix (which is in row echelon) form is returned.

In [26]:
from math import floor, ceil
from sys import float_info

def find_row_echelon_form(matrix: list):
    rows = len(matrix)
    columns = len(matrix[0])

    pivot = 0
    for row in range(rows):
        if pivot >= columns:
            break

        # When pivot is 0 find a row where pivot element is not 0 and
        if matrix[row][pivot] == 0:
            for r in range(row + 1, rows):
                if matrix[r][pivot] != 0:
                    # swap current row and row with non-zero pivot element
                    matrix[row], matrix[r] = matrix[r], matrix[row]
                    break
                else:
                    pivot += 1
                    continue

        # Now, when pivot position is not 0, do elementary row transformation
        # so as to ensure below the pivot we get zeros in the entire column
        for r in range(row + 1, rows):
            factor_nr = float(matrix[r][pivot])
            factor_dr = float(matrix[row][pivot])
            try:
                factor = (factor_nr / factor_dr)
            except ZeroDivisionError:
                print("Error: Division by zero!" + "Please take another suitable matrix.")
                print("factor = matrix[r={r}][pivot={pivot}] / matrix[row={row}][[pivot={pivot}] = {nr} / {dr} = {factor}".format(r=r, row=row, pivot=pivot, nr=matrix[r][pivot], dr=matrix[row][pivot], factor=factor))

            for c in range(pivot, columns):
                matrix[r][c] = (float(matrix[r][c]) - factor * float(matrix[row][c]))
                if matrix[r][c] <= sys.float_info.epsilon:
                    matrix[r][c] = 0

        pivot += 1

    return matrix

In [34]:
ref_A_b = find_row_echelon_form(augmented_matrix_A_b)

print_matrix(ref_A_b)

[[618, 909, 905, 407],
[0, 67.67961165048541, 0, 0],
[0, 0, 355.74110032362455, 143.789644012945]]


`is_row_echelon()` function checks if its input matrix is in row echelon form. It verifies that each row has a non-zero pivot element, and all the elements below the pivot in the same column are zero.

Start by iterating over each row in the matrix.
- if the pivot value is greater than or equal to the number of columns, break out of the loop since all the remaining columns are zero.
- if the pivot element in the current row is zero the matrix is not in row echelon form, so return False
- when `pivot element` is non-zero then iterate over the rows below the current row, and if all the elements below the pivot in the same column are zero
Inside the inner loop, it checks if the element in the current row and the pivot column is non-zero. If it is, it means that the matrix is not in row echelon form, and the function returns False.

After checking all the elements below the pivot in the current column, the pivot value is incremented to move to the next column.

Once the outer loop finishes iterating over all the rows, it means that the matrix satisfies the conditions for row echelon form, and the function returns True.

In [35]:
def is_row_echelon(matrix: list):
    rows = len(matrix)
    columns = len(matrix[0])

    pivot = 0
    for row in range(rows):
      if pivot >= columns:
        break

      # Pivot positions cannot have 0 in a matrix which is in row echelon form
      if matrix[row][pivot] < float_info.epsilon:
        return False

      # All elements below the pivot must be 0
      for r in range(row + 1, rows):
        if matrix[r][pivot] != 0:
          return False

      pivot += 1

    return True

In [36]:
print("Given matrix is in REF") if is_row_echelon(ref_A_b) else print("Given matrix is not in REF")

Given matrix is in REF


Function to get reduced row echelon form of a matrix in row echelon form

In [37]:
def find_reduced_row_echelon_form(matrix: list, find_ref: bool=False):
    if not is_row_echelon(matrix) and find_ref:
        matrix = find_row_echelon_form(matrix)
    elif not is_row_echelon(matrix) and not find_ref:
        print("Given matrix is not in row echelon form, choose a different matrix to operate on!")

    rows = len(matrix)
    columns = len(matrix[0])

    pivot = 0
    for row in range(rows):
        if pivot >= columns:
            break

        # Divide the pivot row by the pivot element
        pivot_element = matrix[row][pivot]
        for c in range(columns):
          try:
            matrix[row][c] /= pivot_element
          except ZeroDivisionError:
            print("Error: Division by zero!" + "Please take another suitable matrix.")
            break

        # Eliminate non-zero elements above the pivot
        for r in range(row):
            factor = matrix[r][pivot]
            for c in range(columns):
                matrix[r][c] -= factor * matrix[row][c]

        # Eliminate non-zero elements below the pivot
        for r in range(row + 1, rows):
            factor = matrix[r][pivot]
            for c in range(columns):
                matrix[r][c] -= factor * matrix[row][c]

                if matrix[r][c] <= sys.float_info.epsilon:
                  matrix[r][c] = 0

        pivot += 1

    return matrix


In [38]:
rref_A_b = find_reduced_row_echelon_form(ref_A_b)

print_matrix(rref_A_b)

[[1.0, 0, 0, 0.0666687893453658],
[0, 1.0, 0, 0],
[0, 0, 1.0, 0.40419744550780545]]


1/c) Get a 5 x 7 matrix and a 5 x 1 matrix

In [39]:
five_by_seven_A = generate_random_matrix(rows=5, columns=7)
five_by_one_b = generate_random_matrix(rows=5, columns=1)

print("Matrix A: ")
print_matrix(five_by_seven_A)
print("\n\n\n\n\n")

print("Matrix b: ")
print_matrix(five_by_one_b)

Matrix A: 
[[987, 640, 188, 583, 854, 400, 162],
[226, 736, 403, 357, 240, 625, 682],
[468, 361, 266, 792, 844, 649, 143],
[911, 307, 155, 543, 265, 526, 364],
[130, 698, 129, 662, 743, 820, 134]]






Matrix b: 
[[334],
[696],
[332],
[312],
[268]]


In [40]:
aug_mtx_A_b = construct_augmented_matrix(five_by_seven_A, five_by_one_b)

print_matrix(aug_mtx_A_b)

[[987, 640, 188, 583, 854, 400, 162, 334],
[226, 736, 403, 357, 240, 625, 682, 696],
[468, 361, 266, 792, 844, 649, 143, 332],
[911, 307, 155, 543, 265, 526, 364, 312],
[130, 698, 129, 662, 743, 820, 134, 268]]


In [41]:
ref_A_b = find_row_echelon_form(aug_mtx_A_b)

print_matrix(ref_A_b)

[[987, 640, 188, 583, 854, 400, 162, 334],
[0, 589.4549138804458, 359.95238095238096, 223.5065856129686, 44.45390070921985, 533.4093211752786, 644.9057750759879, 619.5217831813577],
[0, 0, 141.72325332764976, 493.7464918733843, 434.72481574170837, 407.2698369864144, 3.2380610252461253, 113.15948655189484],
[0, 0, 0, 4.891590678824741, 0, 156.8004052684904, 214.4741641337386, 3.718338399189463],
[0, 0, 0, 0, 584.2350668280072, 0, 0, 0]]


In [42]:
print("Given matrix is in REF") if is_row_echelon(ref_A_b) else print("Given matrix is not in REF")

Given matrix is in REF


In [43]:
rref_A_b = find_reduced_row_echelon_form(ref_A_b, True)

print_matrix(rref_A_b)

[[1.0, 0, 0, 0, 0, -33.592152188686185, -47.04794082000331, -0.9853294811826622],
[0, 1.0, 0, 0, 0, 55.1908949981136, 77.73365439193574, 1.8923719537975445],
[0, 0, 1.0, 0, 0, -108.80233582643422, -152.72946162608662, -1.8498127411521825],
[0, 0, 0, 1.0, 0, 32.055095277547515, 43.84548467274216, 0.760149130074562],
[0, 0, 0, 0, 1.0, 0, 0, 0]]


In [57]:
twelve_by_twenty_four = generate_random_matrix(12, 24, False)

print_matrix(twelve_by_twenty_four)

[[62085, 85389, 77852, 31339, 72606, 25965, 43922, 84704, 49640, 90981, 99777, 23287, 36939, 32248, 46114, 96450, 54519, 61587, 54677, 30341, 32133, 82117, 25493, 21602],
[65435, 89459, 28784, 91416, 66416, 64745, 95366, 17892, 54924, 42663, 72976, 55861, 60661, 96610, 44544, 12873, 25568, 59457, 10698, 90637, 60212, 77556, 45668, 14497],
[30193, 27116, 34749, 17079, 65538, 41873, 10999, 53878, 36331, 18464, 63453, 77010, 68290, 60126, 35784, 12855, 59579, 58341, 34880, 58348, 47593, 79883, 35444, 17161],
[80590, 60301, 93076, 54785, 29239, 72932, 16547, 61899, 13120, 86403, 70285, 53115, 53883, 76980, 88460, 41294, 51017, 28958, 81100, 71102, 88883, 34320, 58836, 64308],
[81468, 81050, 81165, 21197, 12364, 38603, 85255, 23471, 96924, 72062, 43848, 85848, 12132, 20130, 67774, 99381, 68763, 52553, 53500, 59955, 89885, 83669, 85852, 47407],
[83935, 50274, 83894, 63495, 50672, 85693, 21715, 22525, 90253, 62970, 47849, 52554, 81514, 30635, 86628, 26811, 71683, 86905, 68356, 33477, 40075, 4

In [58]:
twelve_by_twenty_four_ref = find_row_echelon_form(twelve_by_twenty_four)

print_matrix(twelve_by_twenty_four_ref)

[[62085, 85389, 77852, 31339, 72606, 25965, 43922, 84704, 49640, 90981, 99777, 23287, 36939, 32248, 46114, 96450, 54519, 61587, 54677, 30341, 32133, 82117, 25493, 21602],
[0, 0, 0, 63753.04153982444, 0, 64278.845131674316, 0, 0, 0, 0, 0, 0, 0, 59246.79452363695, 30375.917596843043, 0, 37617.194636385604, 0, 0, 0, 0, 0, 0, 70917.61161311106],
[0, 0, 0, 21126.65072078602, 0, 0, 0, 0, 23142.854232101163, 0, 0, 21071.446323588632, 31574.80430055569, 0, 377.7511240470121, 0, 0, 3643.2645566561987, 0, 0, 0, 0, 10406.073608762184, 0],
[0, 0, 0, 1838.3069662559392, 30228.457308528632, 0, 0, 12684.992478054279, 0, 0, 14929.73413868084, 54586.00465685534, 33694.286965156396, 17486.966346637262, 0, 0, 15950.295254264482, 26471.140662205133, 8289.637416445195, 43592.65147781267, 31966.171152452283, 39948.09493436418, 17565.044341570952, 0],
[0, 0, 0, 0, 0, 0, 27620.507030683744, 0, 3.637978807091713e-12, 0, 0, 26349.47910676374, 0, 0, 4602.805183336084, 0, 0, 0, 0, 20141.509011838607, 47720.044793

In [59]:
twelve_by_twenty_four_rref = find_reduced_row_echelon_form(twelve_by_twenty_four_ref)

print_matrix(twelve_by_twenty_four_rref)

Given matrix is not in row echelon form, choose a different matrix to operate on!
Error: Division by zero!Please take another suitable matrix.
Error: Division by zero!Please take another suitable matrix.
Error: Division by zero!Please take another suitable matrix.
Error: Division by zero!Please take another suitable matrix.
Error: Division by zero!Please take another suitable matrix.
Error: Division by zero!Please take another suitable matrix.
Error: Division by zero!Please take another suitable matrix.
Error: Division by zero!Please take another suitable matrix.
Error: Division by zero!Please take another suitable matrix.
[[1.0, -29993753594.95774, 592690349246212.8, 0, 389368011909704.9, -88405.9006515026, 0, 587703676.622002, -29019.374217782446, 880153361.3807771, 927265.7894638001, -49466384265.75326, 2053110.511597114, 1004605.2375838136, -13073387864.340942, 495278085.3406897, -922510561789.9012, -15685420344.870337, -1539005720.7708797, -37811906358.6736, -11713084093581.977, 2